[![View notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/blob/main/examples/PySDM_examples/Bulenok_2023_MasterThesis/performance_comparison_Srivastava_Setup.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/examples/PySDM_examples/Bulenok_2023_MasterThesis/performance_comparison_Srivastava_Setup.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/Bulenok_2023_MasterThesis/performance_comparison_Srivastava_Setup.ipynb)

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [ ]:
import os
os.environ['NUMBA_NUM_THREADS'] = '10'

from datetime import datetime
import numba  # pylint: disable=unused-import

from PySDM.backends import GPU, CPU
from PySDM.physics import si

from PySDM_examples.Bulenok_2023_MasterThesis.utils import go_benchmark, process_results, plot_processed_results, write_to_file
from PySDM_examples.Bulenok_2023_MasterThesis.setups import setup_coalescence_only_sim, setup_breakup_only_sim, setup_coalescence_breakup_sim

In [ ]:
TIMESTAMP = str(datetime.now().strftime("%Y-%d-%m_%Hh-%Mm-%Ss"))

In [ ]:
SIM_RUN_FILENAME="env_name_" + TIMESTAMP

In [ ]:
assert not os.path.isfile(SIM_RUN_FILENAME)

!echo NUMBA_DEFAULT_NUM_THREADS: $numba.config.NUMBA_DEFAULT_NUM_THREADS >> $SIM_RUN_FILENAME
!echo NUMBA_NUM_THREADS: $numba.config.NUMBA_NUM_THREADS >> $SIM_RUN_FILENAME
!lscpu >> $SIM_RUN_FILENAME
!nvidia-smi >> $SIM_RUN_FILENAME
!nvidia-smi -L >> $SIM_RUN_FILENAME

!cat /proc/cpuinfo >> $SIM_RUN_FILENAME

In [ ]:
CI = 'CI' in os.environ

exponents = [3, 5, 8, 10, 12, 14, 16, 18, 20, 22, 24] if not CI else [3, 5]
n_sds = [2 ** i for i in exponents]

numba_n_threads = [1, 2, 4, 5, 6, 8, 10] if not CI else [1, 2]

n_realisations = 3 if not CI else 2
seeds = list(range(n_realisations))

n_steps_short = 100 if not CI else 3
n_steps_full = 2048 if not CI else 3

# Benchmark regular setup (without scaling)

### Coalescence-only

In [ ]:
res_coalescence_only = go_benchmark(
    setup_coalescence_only_sim, n_sds, n_steps_short, seeds, numba_n_threads=numba_n_threads, double_precision=True, 
    sim_run_filename=SIM_RUN_FILENAME + '-coalescence',
    backends=[CPU, GPU]
)

In [ ]:
coalescence_only_processed = process_results(res_coalescence_only)

In [ ]:
plot_processed_results(coalescence_only_processed, plot_title=f'coalescence-only (n_steps: {n_steps_short})')

In [ ]:
filename=f"{SIM_RUN_FILENAME}-results-coalescence-double-n_steps{n_steps_short}.txt"
write_to_file(filename=filename, d=coalescence_only_processed)

### Breakup-only

In [ ]:
res_breakup_only = go_benchmark(
    setup_breakup_only_sim, n_sds, n_steps_short, seeds, numba_n_threads=numba_n_threads, double_precision=True,
    sim_run_filename=SIM_RUN_FILENAME + '-breakup',
    backends=[CPU, GPU]
)

In [ ]:
breakup_only_processed = process_results(res_breakup_only)

In [ ]:
plot_processed_results(breakup_only_processed, plot_title=f'breakup-only  (n_steps: {n_steps_short})')

In [ ]:
filename=f"{SIM_RUN_FILENAME}-results-breakup-double-n_steps{n_steps_short}.txt"
write_to_file(filename=filename, d=breakup_only_processed)

### Coalescence and Breakup

In [ ]:
res_coal_breakup = go_benchmark(
    setup_coalescence_breakup_sim, n_sds, n_steps_full, seeds, numba_n_threads=numba_n_threads, double_precision=True, 
    sim_run_filename=SIM_RUN_FILENAME + '-coal-break',
    backends=[CPU, GPU]
)

In [ ]:
coal_breakup_processed = process_results(res_coal_breakup)

In [ ]:
plot_processed_results(coal_breakup_processed, plot_title=f'coalescence+breakup (n_steps: {n_steps_full})')

In [ ]:
filename=f"{SIM_RUN_FILENAME}-results-coal_with_breakup-double-n_steps{n_steps_full}.txt"
write_to_file(filename=filename, d=coal_breakup_processed)

# Benchmark setup with scaling

In [ ]:
def total_number_from_n_sd(n_sd):
    return n_sd * 1e8

def dv_from_n_sd(n_sd):
    return n_sd * (0.125 * si.m**3)

### Coalescence-only

In [ ]:
res_coalescence_only_scaled = go_benchmark(
    setup_coalescence_only_sim, n_sds, n_steps_short, seeds, numba_n_threads=numba_n_threads, double_precision=True, 
    sim_run_filename=SIM_RUN_FILENAME + '-coalescence-scaled',
    total_number=total_number_from_n_sd,
    dv=dv_from_n_sd,
    backends=[CPU, GPU]
)

In [ ]:
coalescence_only_processed_scaled = process_results(res_coalescence_only_scaled)

In [ ]:
plot_processed_results(coalescence_only_processed_scaled, plot_title=f'coalescence-only with scaling (n_steps: {n_steps_short})')

In [ ]:
filename=f"{SIM_RUN_FILENAME}-results-scaled-coalescence-double-n_steps{n_steps_short}.txt"
write_to_file(filename=filename, d=coalescence_only_processed_scaled)

### Breakup-only

In [ ]:
res_breakup_only_scaled = go_benchmark(
    setup_breakup_only_sim, n_sds, n_steps_short, seeds, numba_n_threads=numba_n_threads, double_precision=True,
    sim_run_filename=SIM_RUN_FILENAME + '-breakup-scaled',
    total_number=total_number_from_n_sd,
    dv=dv_from_n_sd,
    backends=[CPU, GPU]
)

In [ ]:
breakup_only_processed_scaled = process_results(res_breakup_only_scaled)

In [ ]:
plot_processed_results(breakup_only_processed_scaled, plot_title=f'breakup-only with scaling (n_steps: {n_steps_short})')

In [ ]:
filename=f"{SIM_RUN_FILENAME}-results-scaled-breakup-double-n_steps{n_steps_short}.txt"
write_to_file(filename=filename, d=breakup_only_processed_scaled)

### Coalescence and Breakup

In [ ]:
res_coal_breakup_scaled = go_benchmark(
    setup_coalescence_breakup_sim, n_sds, n_steps_full, seeds, numba_n_threads=numba_n_threads, double_precision=True, 
    sim_run_filename=SIM_RUN_FILENAME + '-coal-break-scaled',
    total_number=total_number_from_n_sd,
    dv=dv_from_n_sd,
    backends=[CPU, GPU]
)

In [ ]:
coal_breakup_processed_scaled = process_results(res_coal_breakup_scaled)

In [ ]:
plot_processed_results(coal_breakup_processed_scaled, plot_title=f'coalescence+breakup with scaling (n_steps: {n_steps_full})')

In [ ]:
filename=f"{SIM_RUN_FILENAME}-results-scaled-coal_with_breakup-double-n_steps{n_steps_full}.txt"
write_to_file(filename=filename, d=coal_breakup_processed_scaled)